# Storing/loading data to/from Richmol files

Calculated field-free rotational and ro-vibrational enegries together with the matrix elements of Cartesian tensor operators (e.g., dipole moment, polarizability) can be stored in Richmol HDF5 files.

Here we demonstrate how to store data into Richmol files and how to load it using `rchm` module.

In [1]:
from richmol import rchm

To start, generate some field-free rotational energies and matrix elements of dipole moment and polarizability for camphor molecule. For details, see "Rotational dynamics Quickstart":

In [2]:
from richmol.rot import solve
from richmol.rot import LabTensor
from richmol.rot import Molecule

camphor = Molecule()
camphor.XYZ = ("angstrom",
    "O",     -2.547204,    0.187936,   -0.213755,
    "C",     -1.382858,   -0.147379,   -0.229486,
    "C",     -0.230760,    0.488337,    0.565230,
    "C",     -0.768352,   -1.287324,   -1.044279,
    "C",     -0.563049,    1.864528,    1.124041,
    "C",      0.716269,   -1.203805,   -0.624360,
    "C",      0.929548,    0.325749,   -0.438982,
    "C",      0.080929,   -0.594841,    1.638832,
    "C",      0.791379,   -1.728570,    0.829268,
    "C",      2.305990,    0.692768,    0.129924,
    "C",      0.730586,    1.139634,   -1.733020,
    "H",     -1.449798,    1.804649,    1.756791,
    "H",     -0.781306,    2.571791,    0.321167,
    "H",      0.263569,    2.255213,    1.719313,
    "H",      1.413749,   -1.684160,   -1.316904,
    "H",     -0.928638,   -1.106018,   -2.110152,
    "H",     -1.245108,   -2.239900,   -0.799431,
    "H",      1.816886,   -1.883799,    1.170885,
    "H",      0.276292,   -2.687598,    0.915376,
    "H",     -0.817893,   -0.939327,    2.156614,
    "H",      0.738119,   -0.159990,    2.396232,
    "H",      3.085409,    0.421803,   -0.586828,
    "H",      2.371705,    1.769892,    0.297106,
    "H",      2.531884,    0.195217,    1.071909,
    "H",      0.890539,    2.201894,   -1.536852,
    "H",      1.455250,    0.830868,   -2.487875,
    "H",     -0.267696,    1.035608,   -2.160680)

camphor.dip = [1.21615, -0.30746, 0.01140]

camphor.pol = [[115.80434, -0.58739, 0.03276], \
               [-0.58739, 112.28245, 1.36146], \
               [0.03276, 1.36146, 108.47809]]

camphor.frame = "diag(pol)"
camphor.sym = "D2"
sol = solve(camphor, Jmin=0, Jmax=5, verbose=False)

dip = LabTensor(camphor.dip, sol)
pol = LabTensor(camphor.pol, sol)

We can now store molecular information and matrix elements of Cartesian tensor operators into HDF5 file

In [3]:
rchm.store('camphor.h5', camphor, comment="S-Camphor structure from the Supersonic expansion FTMW spectra, Kisiel, et al., PCCP 5, 820 (2003)", replace=True)

rchm.store('camphor.h5', dip, comment="S-Camphor dipole moment (in atomic units), calculated using DFT", replace=True)

rchm.store('camphor.h5', pol, comment="S-Camphor polarizability (in atomic units), calculated using DFT", replace=True)

To store the field-free solutions, they must first be transformed into a Cartesian tensor form

In [4]:
h0 = LabTensor(camphor, sol)

rchm.store('camphor.h5', h0, comment="S-Camphor field-free energies (in cm^-1)", replace=True)

To inspect which data groups are stored in file, use function `inspect_file`

In [5]:
data_groups = rchm.inspect_file('camphor.h5')

for name, obj in data_groups.items():
    print("data group:", name)
    for elem in vars(obj).keys():
        print("\tattribute:", elem)

data group: dip
	attribute: Jlist1
	attribute: Jlist2
	attribute: Us
	attribute: Ux
	attribute: assign_k1
	attribute: assign_k2
	attribute: assign_m1
	attribute: assign_m2
	attribute: cart
	attribute: comment
	attribute: date
	attribute: dim1
	attribute: dim2
	attribute: dim_k1
	attribute: dim_k2
	attribute: dim_m1
	attribute: dim_m2
	attribute: os
	attribute: rank
	attribute: symlist1
	attribute: symlist2
	attribute: tens_flat
	attribute: __module__
	attribute: __dict__
	attribute: __weakref__
	attribute: __doc__
data group: h0
	attribute: Jlist1
	attribute: Jlist2
	attribute: assign_k1
	attribute: assign_k2
	attribute: assign_m1
	attribute: assign_m2
	attribute: cart
	attribute: comment
	attribute: date
	attribute: dim1
	attribute: dim2
	attribute: dim_k1
	attribute: dim_k2
	attribute: dim_m1
	attribute: dim_m2
	attribute: os
	attribute: rank
	attribute: symlist1
	attribute: symlist2
	attribute: __module__
	attribute: __dict__
	attribute: __weakref__
	attribute: __doc__
data group: m

Here is an example of how to aceess the molecular data stored in 'molecule' data group

In [9]:
mol = data_groups['molecule']

print("date:", mol.date)
print("comment:", mol.comment)
print("polarizability:\n", mol.pol)

print("original polarizability:\n", camphor.pol)

date: 2021-04-15 19:01:37
comment: b'S-Camphor structure from the Supersonic expansion FTMW spectra, Kisiel, et al., PCCP 5, 820 (2003)'
polarizability:
 [[ 1.08035306e+02 -8.04509573e-15 -3.48975965e-15]
 [-1.70532612e-14  1.12624870e+02  5.06130530e-15]
 [-5.09201926e-15  8.38035169e-15  1.15904704e+02]]
original polarizability:
 [[ 1.08035306e+02 -8.04509573e-15 -3.48975965e-15]
 [-1.70532612e-14  1.12624870e+02  5.06130530e-15]
 [-5.09201926e-15  8.38035169e-15  1.15904704e+02]]
